In [ ]:
# training imports
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
from wandb.integration.sb3 import WandbCallback
import wandb
import gymnasium as gym
# general imports
import argparse
import os
from lazywrapper import LazyWrapper, LastObservationWrapper

# define function
def make_custom_env(original_env, n_envs, seed, loaded_default_policy, penalty):
    def _init():
        env = LazyWrapper(LastObservationWrapper(gym.make(original_env)), default_policy=loaded_default_policy, penalty = penalty)
        return env
    return _init
# set parameters
default_model = PPO.load("baselines/pretrained_policies/ppo_CartPole-v1_0")
seed = 0
penalty = -1
original_env = "CartPole-v1"
n_envs = 4
# make environment
vec_env = make_vec_env(env_id=make_custom_env(original_env, n_envs, seed, default_model, penalty), n_envs=n_envs)
vec_obs = vec_env.reset()
# make original environment
original_env = make_vec_env("CartPole-v1", n_envs=4)
obs = original_env.reset()

In [ ]:
original_env.step([0,1,0,1])

In [ ]:
vec_env.step([2,2,2,2])